In [5]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import time
import matplotlib.pyplot as plt
# from sklearn_evaluation import plot

print("Imports complete")

Imports complete


In [ ]:
# Load the data from the text file, ignoring the first line

filename="./data/MiniBooNE_PID.txt"
data=pd.read_csv(filename,delim_whitespace=True,skiprows=1,header=None)
print(data.head(1))
X=data

In [ ]:
""" Create target vector for each row of the dataset. 
The first value in the first row contains the number of signal events, the second the number of background events. 
The target vector should contain 1 for signal events and 0 for background events."""

# Get number of signal and background events
signal, background = open(filename).readline().split()

y = np.concatenate((np.ones(int(signal)), np.zeros(int(background))))
print(y.shape)

Standardization scales the input features to have a mean of 0 and a standard deviation of 1

In [ ]:
# Standardize the input data
scaler = preprocessing.StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
""" save the scaled matrix and the target vector as npy files """
np.save('X.npy', X_scaled)
np.save('y.npy', y)

In [7]:
X=np.load('X.npy') # load the scaled matrix
y=np.load('y.npy')

In [8]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Create a vanilla neural network classifier and train it on the training data
nn_clf = MLPClassifier(random_state=42)

# Define the hyperparameter space to search over for the vanilla neural network classifier
nn_param_grid = {
    'hidden_layer_sizes': [(5,), (10,)],
    'solver': ['adam'],
    'activation': ["relu", "logistic", "tanh", "identity"],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'random_state': [42]
}

# Set up the grid search using 5-fold cross-validation
# nn_grid_search = GridSearchCV(nn_clf, nn_param_grid, cv=5, scoring='accuracy')
nn_grid_search = RandomizedSearchCV(nn_clf, nn_param_grid, cv=5, scoring='accuracy', n_iter=2)

# Perform the grid search on the training set
nn_start_time = time.time()
nn_grid_search.fit(X_train, y_train)
nn_end_time = time.time()
print('Time required to grid search Vanilla Neural Network classifier:', nn_end_time - nn_start_time)

# Print the best hyperparameters and the best cross-validation accuracy
print('Vanilla Neural Network best hyperparameters:', nn_grid_search.best_params_)
print('Vanilla Neural Network best cross-validation accuracy:', nn_grid_search.best_score_)

c:\Anaconda3\envs\ml_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Anaconda3\envs\ml_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Anaconda3\envs\ml_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Anaconda3\envs\ml_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Anaconda3\envs\ml_env\lib\site-packages\sklearn\neural_network\_multi

In [ ]:

# Re-train the classifier using the best hyperparameters on the full training set
nn_clf = nn_grid_search.best_estimator_

# nn_start_time = time.time()
# nn_clf.fit(X_train, y_train)
# nn_end_time = time.time()

# Evaluate the classifier on the test set
# nn_test_accuracy = nn_clf.score(X_test, y_test)
nn_test_accuracy = nn_grid_search.score(X_test, y_test)
print('Time required to train Vanilla Neural Network classifier:', nn_end_time - nn_start_time)
print('Vanilla Neural Network test accuracy:', nn_test_accuracy)

In [4]:
""" Plot the scores for the searched parameters on the vanilla neural network classifier """
plot.grid_search(nn_grid_search.cv_results_, change='learning_rate_init', kind='bar')

NameError: name 'plot' is not defined

In [ ]:
# Create a SVM classifier and train it on the training data
svm_clf = SVC(random_state=42)

svm_param_grid = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10]
}

svm_grid_search = GridSearchCV(svm_clf, svm_param_grid, cv=5, scoring='accuracy')

svm_start_time = time.time()
svm_grid_search.fit(X_train, y_train)
svm_end_time = time.time()

print('SVM best hyperparameters:', svm_grid_search.best_params_)
print('SVM best cross-validation accuracy:', svm_grid_search.best_score_)

svm_clf = svm_grid_search.best_estimator_

# svm_clf.fit(X_train, y_train)

svm_test_accuracy = svm_grid_search.score(X_test, y_test)
print('Time required to train SVM classifier:', svm_end_time - svm_start_time)
print('SVM test accuracy:', svm_test_accuracy)

In [ ]:
# Create a random forest classifier and train it on the training data
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

param_grid = {
    "n_estimators": [10, 50, 100, 200],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 4, 8],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", None]
}

rf_grid_search = GridSearchCV(rf_clf, svm_param_grid, cv=5, scoring='accuracy')

rf_start_time = time.time()
rf_grid_search.fit(X_train, y_train)
rf_end_time = time.time()

print('RF best hyperparameters:', rf_grid_search.best_params_)
print('RF best cross-validation accuracy:', rf_grid_search.best_score_)

rf_clf = rf_grid_search.best_estimator_
# rf_clf.fit(X_train, y_train)

rf_test_accuracy = rf_grid_search.score(X_test, y_test)
print('Time required to train RF classifier:', rf_end_time - rf_start_time)
print('RF test accuracy:', rf_test_accuracy)

In [ ]:
# Generate predictions for the test set
nn_predictions = nn_clf.predict(X_test)
svm_predictions = svm_clf.predict(X_test)
rf_predictions = rf_clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

# Define a dictionary that maps model names to their hyperparameters
model_hyperparams = {
    'Vanilla Neural Network': nn_grid_search.best_params_,
    'SVM': svm_grid_search.best_params_,
    'Random Forest': rf_grid_search.best_params_
}

# Compute the confusion matrix for each model
nn_confusion_matrix = metrics.confusion_matrix(y_test, nn_predictions)
svm_confusion_matrix = metrics.confusion_matrix(y_test, svm_predictions)
rf_confusion_matrix = metrics.confusion_matrix(y_test, rf_predictions)

# Print the confusion matrix and hyperparameters for each model
print('Vanilla Neural Network confusion matrix:')
print(nn_confusion_matrix)
print('Vanilla Neural Network hyperparameters:', model_hyperparams['Vanilla Neural Network'])

print('SVM confusion matrix:')
print(svm_confusion_matrix)
print('SVM hyperparameters:', model_hyperparams['SVM'])

print('Random Forest confusion matrix:')
print(rf_confusion_matrix)
print('Random Forest hyperparameters:', model_hyperparams['Random Forest'])

In [ ]:
# Evaluate the performance of each classifier
nn_accuracy = metrics.accuracy_score(y_test, nn_predictions)
svm_accuracy = metrics.accuracy_score(y_test, svm_predictions)
rf_accuracy = metrics.accuracy_score(y_test, rf_predictions)
print("Vanilla Neural Network Accuracy:", nn_accuracy)
print("SVM Accuracy:", svm_accuracy)
print("Random Forest Accuracy:", rf_accuracy)